#

## Scraping.ipynb

---

Notebook tem como objetivo extrair dados de partidas do site https://football.esportsbattle.com/en/ e salvar em um arquivo .csv

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import requests


##### gerar_url()

Função que recebe uma data de referência e gera uma lista de URLs que busca os torneios

In [3]:
def gerar_url(data_inicial, data_final = datetime.now()):

    datas = [
        [(data_inicial + timedelta(days=n)).year,
        (data_inicial + timedelta(days=n)).month,
        (data_inicial + timedelta(days=n)).day]
        for n in range((data_final - data_inicial).days + 1)
    ]   

    datas_url = []
    pag = 1 
    for i in range(len(datas)):
        ano = datas[i][0]
        mes = datas[i][1]
        dia = datas[i][2]
        if mes in [1,3,5,7,8,10,12] and dia == 31:
            datas_url.append(f"https://football.esportsbattle.com/api/tournaments?page={pag}&dateFrom={ano}%2F{mes}%2F{dia}+11%3A00&dateTo={ano}%2F{mes}%2F{1}+02%3A59")
        elif mes == 2 and dia == 28:
            datas_url.append(f"https://football.esportsbattle.com/api/tournaments?page={pag}&dateFrom={ano}%2F{mes}%2F{dia}+11%3A00&dateTo={ano}%2F{mes}%2F{1}+02%3A59")
        else:
            datas_url.append(f"https://football.esportsbattle.com/api/tournaments?page={pag}&dateFrom={ano}%2F{mes}%2F{dia}+11%3A00&dateTo={ano}%2F{mes}%2F{dia+1}+02%3A59")


    return datas_url

##### Função que pega todos os torneios para a data definida

In [4]:
def pega_torneios(datas_url):
    lista_torneios = []  # Lista de todos os torneios disponíveis por página

    for url in datas_url:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Lança erro se status != 200
            json_torneios = response.json()

            paginas = json_torneios.get('totalPages', 0)
            torneios = json_torneios.get('tournaments', [])

            for pag in range(paginas):
                for torneio in torneios:
                    lista_torneios.append([pag + 1, torneio])

        except Exception as e:
            print(f"[AVISO] Erro ao processar URL: {url}")
            print(f"Motivo: {e}\n")
            continue

    return lista_torneios


##### Função que pega todas as partidas por torneio

In [5]:
def pega_partidas(lista_torneios):
    
    lista_partidas = []
    for i in range(len(lista_torneios)):
        # pag = lista_torneios[i][0]
        id_torneio = lista_torneios[i][1]['id']

        lista_partidas.append(f'https://football.esportsbattle.com/api/tournaments/{id_torneio}/matches')

    return lista_partidas

In [14]:
def pega_resultados(lista_partidas):
    
    dados = []
    for i in range(len(lista_partidas)):
        response = requests.get(lista_partidas[i]) # Faz a requisição na URL
        json_jogos = response.json() # Retorna o JSON

        for jogo in json_jogos:
            dados.append({
                'id': jogo['id'],
                'data': jogo['date'],
                'campeonato': jogo['tournament']['token_international'],
                'estadio': jogo['console']['token_international'],
                'team_player_1': jogo['participant1']['team']['token_international'],
                'player_1': jogo['participant1']['nickname'],
                'score_player_1': jogo['participant1']['score'],
                'score_player_2': jogo['participant2']['score'],
                'player_2': jogo['participant2']['nickname'],
                'team_player_2': jogo['participant2']['team']['token_international']
            })

    
    df_partidas = pd.DataFrame(dados)
    return df_partidas

In [9]:
urls = gerar_url(datetime(2025,8,7))
torneios = pega_torneios(urls)
partidas = pega_partidas(torneios)
jogos = pega_resultados(partidas)

In [12]:
jogos.dados

AttributeError: 'DataFrame' object has no attribute 'dados'

In [7]:
if __name__ == "__main__":
    # urls = gerar_url(datetime(2025,1,1), datetime(2025,7,31))
    urls = gerar_url(datetime(2025,7,8))
    torneios = pega_torneios(urls)
    partidas = pega_partidas(torneios)
    jogos = pega_resultados(partidas)


KeyboardInterrupt: 

In [ ]:
# jogos.to_csv("../../data/raw/raw_base_jogos.csv", index=False, encoding='utf-8')